In [18]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/cs229/sd_data/')
%cd drive/MyDrive/cs229/sd_data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/cs229/sd_data'
/content/drive/.shortcut-targets-by-id/1hNmGL28ABDHXyODHwa494F88Q-xe1iut/sd_data


### Prepare dataset

In [19]:
import pickle
with open("all_added_emb.pickle", "rb") as f:
    all_added_emb = pickle.load(f)

In [20]:
%%capture
pip install git+https://github.com/openai/CLIP.git

In [21]:
import clip
encoder, preprocess = clip.load("ViT-B/32");

In [35]:
from PIL import Image
import numpy as np
import os
import torch
from torchvision import transforms
from tqdm import tqdm

aug = transforms.Compose([
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
    transforms.GaussianBlur((5, 5)),
])

CLIP_embeddings, TI_embeddings = [], []
"""
CLIP_embeddings is a (D, 512) numpy array containing the CLIP embeddings
of "0.jpeg" for various community-generated concepts.

TI_embeddings is a (D, 768) numpy array containing the corresponding
textual inversion embedding for each concept in CLIP_embeddings.
"""
num_augs = 4
for concept in tqdm(os.listdir('./sd-concepts-library/')):
  concept_path = f"./sd-concepts-library/{concept}/"
  img_path = concept_path + "3.jpeg"
  if os.path.isdir(concept_path) and os.path.isfile(img_path) and concept in all_added_emb:
    img = Image.open(img_path)
    for _ in range(num_augs):
      img_tensor = preprocess(aug(img)).unsqueeze(0)
      CLIP_embeddings.append(encoder.encode_image(img_tensor).squeeze().detach().numpy())
      key = list(all_added_emb[concept].keys())[0]
      TI_embeddings.append(all_added_emb[concept][key].detach().numpy())

CLIP_embeddings = np.stack(CLIP_embeddings)
TI_embeddings = np.stack(TI_embeddings)

100%|██████████| 723/723 [26:27<00:00,  2.20s/it]


In [37]:
pickle.dump(CLIP_embeddings, open("3_AUG_CLIP_embeddings.pickle", "wb"))
pickle.dump(TI_embeddings, open("3_AUG_TI_embeddings.pickle", "wb"))
print(len(CLIP_embeddings))

2476


In [ ]:
# from PIL import Image
# import os

# CLIP_embeddings, TI_embeddings = [], []
# for concept in os.listdir('./sd-concepts-library/'):
#   concept_CLIP_embeddings = []
#   concept_path = f"./sd-concepts-library/{concept}/"
#   if os.path.isdir(concept_path) and concept in all_added_emb:
#     for f in os.listdir(concept_path):
#       if f[-5:] == ".jpeg":
#         img = Image.open(concept_path+f)
#         concept_CLIP_embeddings.append(preprocess(img))
#   if len(concept_CLIP_embeddings) != 0:
#     concept_CLIP_embeddings = encoder.encode_image(torch.stack(concept_CLIP_embeddings))
#     CLIP_embeddings.append(concept_CLIP_embeddings)
#     key = list(all_added_emb[concept].keys())[0]
#     TI_embeddings.append(all_added_emb[concept][key].detach().numpy())